
#### This python notebook is used to update Kibana view with static lookups for CPV and Country fields.
Example:
    CPV: 03000000 -> 03000000 - Agricultural, farming, fishing, forestry and related products
    Country: BE -> BE - Belgium

In [11]:
import json
import requests

from requests.auth import HTTPBasicAuth

from ted_data_eu import config
from ted_data_eu.services.etl_pipelines.ted_data_etl_pipeline import TED_DATA_ETL_PIPELINE_NAME

INDEX = TED_DATA_ETL_PIPELINE_NAME
KIBANA_URL = 'ttps://kibana.ted-data.eu'
KIBANA_VIEW_NAME = 'ted_data'

HEADERS = {'Content-type': 'application/json', 'kbn-xsrf': 'kibana'}
HTTP_AUTH = HTTPBasicAuth(config.ELASTIC_USER, config.ELASTIC_PASSWORD)

In [12]:
def get_kibana_view_id_by_name(view_name: str) -> str:
    """
    Get Kibana view id by name
    :param view_name: name of the view
    :return: id of the view
    """
    response = requests.get(f"{KIBANA_URL}/api/data_views",
                            headers=HEADERS,
                            auth=HTTP_AUTH,
                            )
    response.raise_for_status()
    data_views = response.json()['data_view']
    for data_view in data_views:
        if data_view['title'] == view_name:
            return data_view['id']
    raise Exception(f"Could not find data view with name {view_name}")


DATA_VIEW_ID = get_kibana_view_id_by_name(KIBANA_VIEW_NAME)
DATA_VIEW_ID

'1cf52c93-78e0-4dbd-a5b3-2cdb1f391279'

In [13]:
from typing import List


def update_field_with_static_lookup(field_name: str, field_keys: List[str], key_values: List[str]):
    """
    Updates fields in Kibana view with static lookups
    :param field_name: name of the field to update
    :param field_keys: list of keys to use in the static lookup
    :param key_values: list of values to use in the static lookup
    :return: None
    """
    kibana_request_structure = {
        "fields": {
            field_name: {
                "format": {
                    "id": "static_lookup",
                    "params": {
                        "lookupEntries": [{"key": key, "value": value} for key, value in zip(field_keys, key_values)],
                        "unknownKeyValue": None
                    }
                }
            }
        }
    }
    response = requests.post(url=f"{KIBANA_URL}/api/data_views/data_view/{DATA_VIEW_ID}/fields",
                             headers=HEADERS,
                             auth=HTTP_AUTH,
                             data=json.dumps(kibana_request_structure))
    response.raise_for_status()
    print(f"Updated field {field_name} with static lookup done successfully")

Update CPV Fields with static lookup

In [14]:
from ted_data_eu.services.etl_pipelines.ted_data_etl_pipeline import CPV_RANK_0, CPV_RANK_1, CPV_RANK_2, CPV_RANK_3, \
    CPV_RANK_4, CPV_PARENT, LOT_CPV_COLUMN_NAME
from ted_data_eu.adapters.cpv_processor import CPVProcessor

FIELDS_WITH_CPV = [LOT_CPV_COLUMN_NAME, CPV_RANK_0, CPV_RANK_1, CPV_RANK_2, CPV_RANK_3, CPV_RANK_4, CPV_PARENT]
FIELDS_WITH_CPV = [f"{field}.keyword" for field in FIELDS_WITH_CPV]

cpv_processor = CPVProcessor()

for field_name in FIELDS_WITH_CPV:
    update_field_with_static_lookup(field_name, cpv_processor.get_all_cpvs_name_as_list(),
                                    cpv_processor.get_all_cpvs_label_as_list())

Updated field main_cpvs.keyword with static lookup done successfully
Updated field cpv0.keyword with static lookup done successfully
Updated field cpv1.keyword with static lookup done successfully
Updated field cpv2.keyword with static lookup done successfully
Updated field cpv3.keyword with static lookup done successfully
Updated field cpv4.keyword with static lookup done successfully
Updated field cpv_parent.keyword with static lookup done successfully


Update Country field with static lookup

In [15]:
from ted_data_eu.services.etl_pipelines.ted_data_etl_pipeline import LOT_NUTS_0
import pycountry

country_field_name = f"{LOT_NUTS_0}.keyword"
country_codes = [country.alpha_2 for country in pycountry.countries]
country_names = [country.name for country in pycountry.countries]
update_field_with_static_lookup(country_field_name, country_codes, country_names)

Updated field lots_nuts_0.keyword with static lookup done successfully
